In [ ]:
import names
import random
import pgeocode
import csv
import requests
import json
import pgeocode

## Generate Pinot Table (Prerequisite: Schema has already been created)

In [ ]:
v_path_to_table_config = "trips/trips_realtime_table_config.json"

In [ ]:
response = requests.post('http://pinot-controller.pinot:9000/tables', json=json.load(open(v_path_to_table_config)))
print(response)
print(response.text)

## Kafka Create Topic 

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers="pinot-kafka.pinot:9092", 
    client_id='test'
)

In [ ]:
# deletion of Kafka Topic fails
# admin_client.delete_topics(topics=["trips_generated_data"])

In [ ]:
# trips_generated_data created on 27th March 2021, wrong data (multiple times same records)
# admin_client.create_topics(new_topics=[NewTopic(name="trips_generated_data", num_partitions=1, replication_factor=1)], validate_only=False)

## Kafka Producer - Generate Data

In [ ]:
#Choose random city of file containing German cities with postcode

geocode_file = open('./pgeocode_data/DE.txt')
# skip first line (header)
geocode_file.readline()
geocode_list = list(csv.reader(geocode_file))

def choose_random_city():
    return random.choice(geocode_list)
    
# Generation of License Plate    
def generate_license_plate():
    #create a pool of letters to choose from
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    numbers = '0123456789'

    #create a program loop

     #generate 3 randomly chosen letters, L1, L2, L3
    L1 = random.choice(letters)
    L2 = random.choice(letters)
    L3 = random.choice(letters)
    L4 = random.choice(letters)
     #generate 4 randomly chosen numbers, N1, N2, N3, N4
    N1 = random.choice(numbers)
    N2 = random.choice(numbers)
  
        #combine it together into one print function
    return(L1+L2+'-'+L3+L4+'-'+N1+N2)

# Calculation of price based on distance between start city and end destination
def calculate_price(v_distance):
    v_multiplicator=round(random.uniform(0.8, 2.0),2)
    v_price=round(v_distance*v_multiplicator,2)
    return(v_price)
    

In [ ]:
dist = pgeocode.GeoDistance('de')

In [ ]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers=['pinot-kafka.pinot:9092'], value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [ ]:
# Generate data
for i in range(50000):
    v_start_location=choose_random_city()
    v_end_location=choose_random_city()
    #if travelling in one postcode area
    if v_start_location[1] == v_start_location[2]:
        v_distance=random(0.5,6)
    else:
        v_distance=dist.query_postal_code(v_start_location[1], v_end_location[1])
    v_requesttime = random.randint(1585263600000,1616847444944);
    v_waiting_time_millis = random.randint(1,3600000)
    v_trip_time = round((v_distance/random.randint(45,60)) * 60 *60*1000)
    record = {
        "rider_name": names.get_full_name(),
        "driver_name": names.get_full_name(),
        "license_plate":generate_license_plate(),
        "start_location":v_start_location[2],
        "start_zip_code":v_start_location[1],
        "start_location_state":v_start_location[3],
        "end_location":v_end_location[2],
        "end_zip_code":v_end_location[1],
        "end_location_state":v_end_location[3],
        "rider_is_premium":random.randint(0, 1),
        "count":1,
        "payment_amount": calculate_price(v_distance),
        "payment_tip_amount": random.randint(5,50),
        "trip_wait_time_millis":v_waiting_time_millis,
        "rider_rating":random.randint(0,5),
        "driver_rating":random.randint(0,5),
        "trip_start_time_millis": v_requesttime+v_waiting_time_millis,
        "request_time_millis":v_requesttime,
        "trip_end_time_millis": v_requesttime+v_waiting_time_millis+v_trip_time
    }
    if i % 50 == 0:
        print(f'{i} records generated')
    producer.send('trips_gendata', value=record)
